In [ ]:
from datasets import DatasetDict, load_dataset

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", use_auth_token=True)

In [2]:
common_voice['train'][4]

{'client_id': '0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd00383f604e1e17c38d6ed8adf1bd2ccbf927a52c5adefb8ac4b158ce27a7c2ed9581e71202eb302dfb3',
 'path': '/users/ujan/.cache/huggingface/datasets/downloads/extracted/c97e32e2de32510be7c273dbcdc82db93cb021f851ff1f0751145082d83effd9/common_voice_hi_26010471.mp3',
 'audio': {'path': '/users/ujan/.cache/huggingface/datasets/downloads/extracted/c97e32e2de32510be7c273dbcdc82db93cb021f851ff1f0751145082d83effd9/common_voice_hi_26010471.mp3',
  'array': array([-2.13162821e-14,  1.40332190e-13,  2.23820962e-13, ...,
          1.03580276e-03,  6.28637266e-04,  3.20890395e-04]),
  'sampling_rate': 48000},
 'sentence': 'तुम ने टॉम को कहाँ भेज दिया?',
 'up_votes': 2,
 'down_votes': 1,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'hi',
 'segment': ''}

In [1]:
import jax.numpy as jnp
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import FlaxWhisperForConditionalGeneration
from transformers import WhisperForConditionalGeneration

In [ ]:
#feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
#tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="hindi", task="transcribe")
# We only need to set the task id when the language is specified (i.e. in a multilingual setting)
#tokenizer.set_prefix_tokens(language="hindi", task="transcribe")
#processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="hindi", task="transcribe")

# model
model = FlaxWhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-tiny", # small
    seed=42,
    dtype=getattr(jnp, 'float16')
)

In [9]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny.en")
#tokenizer.save_pretrained("/users/ujan/Downloads/whisper_en_tokenizer")

In [3]:
from transformers import WhisperConfig
config = WhisperConfig.from_pretrained('openai/whisper-tiny')

In [4]:
model = FlaxWhisperForConditionalGeneration.from_pretrained('openai/whisper-tiny')
print(model.config)
model.config.update({"suppress_tokens": []})
print(model.config)

WhisperConfig {
  "_name_or_path": "openai/whisper-tiny",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 384,
  "decoder_attention_heads": 6,
  "decoder_ffn_dim": 1536,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 4,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 6,
  "encoder_ffn_dim": 1536,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 4,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_

In [3]:
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-large-v2')
model.save_pretrained('/home/ujan/speech-processing/models/pretrained_models/whisper-large-v2')

In [13]:
import re
from datasets import load_dataset
import datasets

data_dir = '/home/ujan/Datasets/'
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
text_column = "transcript"
audio_column = "audio"

In [4]:
def path_remap(x):

    # get audio path
    #path_list = x['audio'].split('/')
    path = x['audio']

    #for i in range(len(path_list)):
        #if path_list[i] == 'wav': break

    #new_path = '/'.join(path_list[i:])
    #new_path = args.data_dir+'/'+new_path
    new_path = data_dir+'/'+path
    x['audio'] = new_path

    return x

In [8]:
# data files
data_files = {
    'test': data_dir+'/final_test_v2a.csv', # final_train.csv
}

raw_datasets = load_dataset('csv', data_files=data_files)

# map to new audio path
raw_datasets = raw_datasets.map(path_remap, batched=False)



# remove punctuations
def remove_special_characters(batch):
    batch[text_column] = re.sub(chars_to_ignore_regex, "", batch[text_column]).lower() + " "
    return batch
    

raw_datasets = raw_datasets.map(
    remove_special_characters,
    desc="remove special characters from datasets",
)

Using custom data configuration default-985751aa69de9f8c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/ujan/.cache/huggingface/datasets/csv/default-985751aa69de9f8c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/109251 [00:00<?, ?ex/s]

remove special characters from datasets:   0%|          | 0/109251 [00:00<?, ?ex/s]

In [11]:
raw_datasets['test'][0]

{'audio': '/home/ujan/Datasets//gale2/wav/train/gale2015_2/CCTV1_LEGALREPORT_CMN_20070403_123701_89.wav',
 'transcript': '是不太赞成这种 这种很莽撞的 不负责任的这样的婚姻 ',
 'duration': 5.17999999999995}

In [14]:
raw_datasets = raw_datasets.cast_column(
    audio_column, datasets.features.Audio(sampling_rate=16000)
)

In [16]:
raw_datasets['test'][0]

{'audio': {'path': '/home/ujan/Datasets//gale2/wav/train/gale2015_2/CCTV1_LEGALREPORT_CMN_20070403_123701_89.wav',
  'array': array([-0.00018311,  0.00170898,  0.0012207 , ..., -0.00646973,
         -0.00805664, -0.00961304], dtype=float32),
  'sampling_rate': 16000},
 'transcript': '是不太赞成这种 这种很莽撞的 不负责任的这样的婚姻 ',
 'duration': 5.17999999999995}

In [17]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 109251
    })
})

In [26]:
raw_datasets.push_to_hub("Ujan/asr_testset_zh_16k", private=True)

Pushing split test to the Hub.


  0%|          | 0/4 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: Error while uploading 'data/test-00014-of-00032-b9f5e362471b1a0c.parquet' to the Hub.